In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/vehicle-dataset-from-cardekho/car data.csv")
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
print(df.Transmission.unique())
print(df.Fuel_Type.unique())
print(df.Seller_Type.unique())

In [ ]:
current_year = 2021

In [ ]:
df['years_old'] = current_year - df.Year

In [ ]:
df[df.Present_Price == df.Present_Price.max()]

In [ ]:
df = df.drop(index=86, axis=0)

In [ ]:
df.pivot_table(columns='Fuel_Type', values='Selling_Price', aggfunc=['mean', 'count'])

In [ ]:
df['Petrol'] = pd.get_dummies(df['Fuel_Type']).iloc[:,2]

In [ ]:
df.pivot_table(columns='Transmission', values='Selling_Price', aggfunc=['mean', 'count'])

In [ ]:
df['Manual'] = pd.get_dummies(df['Transmission']).iloc[:,1]

In [ ]:
df

In [ ]:
df.pivot_table(columns='Seller_Type', values='Selling_Price', aggfunc=['mean', 'count'])

In [ ]:
df['Dealer'] = pd.get_dummies(df['Seller_Type']).iloc[:,0]

In [ ]:
df

In [ ]:
df2 = df[['Selling_Price','Present_Price', 'Owner', 'years_old', 'Petrol', 'Manual', 'Dealer']]

In [ ]:
correlation = df2.corr().sort_values(by="Selling_Price", ascending=False)
print(correlation['Selling_Price'])

In [ ]:
df2.drop(['Owner'], axis=1, inplace=True)

In [ ]:
df2

In [ ]:
X = df2.iloc[:,1:]
y = df2.iloc[:,0]

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()

In [ ]:
model.fit(X, y)

In [ ]:
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
feature_importance.nlargest(5).plot(kind='barh')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
parametrs = {
    'n_estimators': [x for x in range(1,10)],
    'max_features': ['sqrt'],
    'max_depth': [x for x in range(1, 10, 1)],
    'min_samples_split': [2,5,10,15,100],
    'min_samples_leaf': [1,2,5,10]
}

In [ ]:
grid_search_reg = GridSearchCV(RandomForestRegressor(), parametrs, scoring = 'neg_mean_squared_error', cv=5, n_jobs=-1)

In [ ]:
grid_search_reg.fit(X_train, y_train)

In [ ]:
grid_search_reg.best_params_

In [ ]:
predictions = grid_search_reg.best_estimator_.predict(X_test)

In [ ]:
import seaborn as sns
sns.distplot(y_test-predictions)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
R2 = metrics.r2_score(y_test,predictions)
R2